<a href="https://colab.research.google.com/github/RomulusGwelt/deep-learning-kbtu-2019/blob/master/CatFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras, datetime
from keras.layers import Input, Dense, Conv2D
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
# from keras.applications import mobilenetv2
import numpy as np


# !pip uninstall tensorflow
# !pip install tensorflow-gpu

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

import numpy as np 
import pandas as pd 

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
import keras
from math import ceil
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.applications.resnet50 import preprocess_input
from keras.applications.xception import preprocess_input
import os 
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from cyclicLR import CyclicLR
from keras.optimizers import Adam, SGD, RMSprop
import datetime
from keras.applications import xception
import cv2
# print(device_lib.list_local_devices())
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from google.colab.patches import cv2_imshow


In [0]:
img_size = 224

mode = 'lmks'

output_size = 18

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

data_00 = np.load('lmks_CAT_00.npy', allow_pickle=True)
data_01 = np.load('lmks_CAT_01.npy', allow_pickle=True)
data_02 = np.load('lmks_CAT_02.npy', allow_pickle=True)
data_03 = np.load('lmks_CAT_03.npy', allow_pickle=True)
data_04 = np.load('lmks_CAT_04.npy', allow_pickle=True)
data_05 = np.load('lmks_CAT_05.npy', allow_pickle=True)
data_06 = np.load('lmks_CAT_06.npy', allow_pickle=True)

print("Loaded data")

x_train = np.concatenate((data_00.item().get('imgs'), data_01.item().get('imgs'), data_02.item().get('imgs'), data_03.item().get('imgs'), data_04.item().get('imgs'), data_05.item().get('imgs')), axis=0)
y_train = np.concatenate((data_00.item().get(mode), data_01.item().get(mode), data_02.item().get(mode), data_03.item().get(mode), data_04.item().get(mode), data_05.item().get(mode)), axis=0)

print("Merged data")



Loaded data
Merged data


In [0]:
x_test = np.array(data_06.item().get('imgs'))
y_test = np.array(data_06.item().get(mode))

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))

y_train = np.reshape(y_train, (-1, output_size))
y_test = np.reshape(y_test, (-1, output_size))

print("Here goes the NN")



Here goes the NN


In [0]:
input_shape = (224, 224, 3)
base_model = xception.Xception(input_shape=input_shape, weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(18)(x)

model = Model(inputs=base_model.input, outputs=predictions)














83689472/83683744 [==============================] - 3s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
print("ALMOST THERE")

# model.summary()
# early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# training
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')


model.fit(x_train, y_train, epochs=50, batch_size=32, shuffle=True,
  validation_data=(x_test, y_test), verbose=1,
  callbacks=[
    # TensorBoard(log_dir='logs/%s' % (start_time)),
    ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto')
  ]
)

ALMOST THERE
Train on 8609 samples, validate on 1388 samples
Epoch 1/50
8609/8609 [==============================] - 201s 23ms/step - loss: 111.4208 - val_loss: 75.5641

Epoch 00001: val_loss improved from inf to 75.56409, saving model to ./models/2019_12_02_22_33_39.h5
Epoch 2/50
8609/8609 [==============================] - 197s 23ms/step - loss: 110.9420 - val_loss: 65.0149

Epoch 00002: val_loss improved from 75.56409 to 65.01486, saving model to ./models/2019_12_02_22_33_39.h5
Epoch 3/50
8609/8609 [==============================] - 198s 23ms/step - loss: 112.5181 - val_loss: 77.6304

Epoch 00003: val_loss did not improve from 65.01486
Epoch 4/50
8609/8609 [==============================] - 198s 23ms/step - loss: 110.2305 - val_loss: 77.4503

Epoch 00004: val_loss did not improve from 65.01486
Epoch 5/50
8609/8609 [==============================] - 198s 23ms/step - loss: 108.7348 - val_loss: 57.4514

Epoch 00005: val_loss improved from 65.01486 to 57.45136, saving model to ./models/

In [0]:
model.save('model_cats.h5')

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
uploaded = drive.CreateFile({'title': 'model_cats.h5'})
uploaded.SetContentFile('model_cats.h5')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1m9qT8ySfH77GyIzEzurVPFcE1JMkNpI-
